### Lab -- Data Prep & Gradient Boosting

Welcome to today's lab!  Today we're going to shift our attention to a more demanding dataset -- the restaurants data.  A quarter million rows, dates, and categorical data make this a more interesting, realistic use case of boosting.  

The point of today's lab will be to experiment with different encoding methods and model parameters.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
import category_encoders as ce 

In [ ]:
# * abcde
# 
abc = '0123456789'
ddd = "0o0o 0O0O0O"

**Step 1:**  Load in your dataset

In [2]:
# your code here

df = pd.read_csv("../../data/restaurants.csv",parse_dates=['visit_date'])


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252108 entries, 0 to 252107
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                252108 non-null  object        
 1   visit_date        252108 non-null  datetime64[ns]
 2   visitors          252108 non-null  int64         
 3   calendar_date     252108 non-null  object        
 4   day_of_week       252108 non-null  object        
 5   holiday           252108 non-null  int64         
 6   genre             252108 non-null  object        
 7   area              252108 non-null  object        
 8   latitude          252108 non-null  float64       
 9   longitude         252108 non-null  float64       
 10  reserve_visitors  108394 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 21.2+ MB


In [86]:
df.head(5)

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors
166836,air_00a91d42b08b08d9,2016-07-01,35,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN
166837,air_00a91d42b08b08d9,2016-07-02,9,2016-07-02,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0
166838,air_00a91d42b08b08d9,2016-07-04,20,2016-07-04,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN
166839,air_00a91d42b08b08d9,2016-07-05,25,2016-07-05,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN
166840,air_00a91d42b08b08d9,2016-07-06,29,2016-07-06,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,NaN


In [ ]:
df.sort_values(by=['id','visit_date'],ascending=True, inplace=True)

In [ ]:
# To sort out issue when try to fit the data, need to sort NaN entries in reserve column

In [91]:
df['reserve_visitors_null'] = (df.reserve_visitors.isnull()).apply(lambda x: 1 if x == True else 0)

In [95]:
df['reserve_visitors'][df.reserve_visitors.isnull()].fillna(0)

Series([], Name: reserve_visitors, dtype: float64)

In [96]:
df['reserve_visitors']

166836     0.0
166837     4.0
166838     0.0
166839     0.0
166840     0.0
          ... 
216643     0.0
216644     0.0
216645     1.0
216646     6.0
216647    37.0
Name: reserve_visitors, Length: 252108, dtype: float64

In [97]:
df.head(5)

,id,visit_date,visitors,calendar_date,day_of_week,holiday,genre,area,latitude,longitude,reserve_visitors,reserve_visitors_null
166836,air_00a91d42b08b08d9,2016-07-01,35,2016-07-01,Friday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1
166837,air_00a91d42b08b08d9,2016-07-02,9,2016-07-02,Saturday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,4.0,0
166838,air_00a91d42b08b08d9,2016-07-04,20,2016-07-04,Monday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1
166839,air_00a91d42b08b08d9,2016-07-05,25,2016-07-05,Tuesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1
166840,air_00a91d42b08b08d9,2016-07-06,29,2016-07-06,Wednesday,0,Italian/French,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,0.0,1


**Step 2:** Create a training and test set.

Make the test set the **last 15 observations for each restaurant**.

Turn each of these variables into `X_train, y_train`, and `X_test, y_test`, respectively.

**Hint:**  This harkens back to our grouping lab -- check this if you forget how to do it.

In [98]:
# your code here
df.sort_values(by=['id','visit_date'],ascending=True, inplace=True)

In [99]:
training = df.groupby('id').apply(lambda x : x.iloc[:-15])
testing =  df.groupby('id').apply(lambda x : x.iloc[-15:])

In [100]:
# Need to drop the date columns as they simply act as an incremental timing,
# so won't be useful for future predictions.
X_train = training.drop(['visitors','visit_date'],axis=1).copy()
y_train = training['visitors'].copy()



X_test = testing.drop(['visitors','visit_date'],axis=1).copy()
y_test = testing['visitors'].copy() 

In [101]:
X_train.head(3)

id calendar_date day_of_week  \
id                                                                            
air_00a91d42b08b08d9 166836  air_00a91d42b08b08d9    2016-07-01      Friday   
                     166837  air_00a91d42b08b08d9    2016-07-02    Saturday   
                     166838  air_00a91d42b08b08d9    2016-07-04      Monday   

                             holiday           genre  \
id                                                     
air_00a91d42b08b08d9 166836        0  Italian/French   
                     166837        0  Italian/French   
                     166838        0  Italian/French   

                                                        area   latitude  \
id                                                                        
air_00a91d42b08b08d9 166836  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     166837  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   
                     166838  Tōkyō-to Chiyoda-ku Kudanminami  35.694003   

                              longitude  reserve_visitors  \
id                                                          
air_00a91d42b08b08d9 166836  139.753595               0.0   
                     166837  139.753595               4.0   
                     166838  139.753595               0.0   

                             reserve_visitors_null  
id                                                  
air_00a91d42b08b08d9 166836                      1  
                     166837                      0  
                     166838                      1

**Step 3:** Experiment with different encoding methods

Let's do a quick check to see how different encoding methods work out of the box on our dataset.

You're going to repeat the same process for each of `OrdinalEncoder`, `TargetEncoder`, and `OneHotEncoder` and see which one gives you the best results on our data.

**3a:** Use an `OrdinalEncoder` to transform your training set with the `fit_transform` method.  Then use the `transform` method to transform your test set.  

**Important:** An important detail here is that the test set is being transformed according to the values in your training set.  

If you are confused about how the transformation is happening, try using the `mapping` attribute on your category encoder to get a hang of what's going on.

In [ ]:
# your code here

In [102]:
ordenc = ce.OrdinalEncoder()
X_train1 = ordenc.fit_transform(X_train)
X_test1 = ordenc.transform(X_test)

In [103]:
X_train1.head()

id  calendar_date  day_of_week  holiday  genre  \
id                                                                            
air_00a91d42b08b08d9 166836   1              1            1        0      1   
                     166837   1              2            2        0      1   
                     166838   1              3            3        0      1   
                     166839   1              4            4        0      1   
                     166840   1              5            5        0      1   

                             area   latitude   longitude  reserve_visitors  \
id                                                                           
air_00a91d42b08b08d9 166836     1  35.694003  139.753595               0.0   
                     166837     1  35.694003  139.753595               4.0   
                     166838     1  35.694003  139.753595               0.0   
                     166839     1  35.694003  139.753595               0.0   
                     166840     1  35.694003  139.753595               0.0   

                             reserve_visitors_null  
id                                                  
air_00a91d42b08b08d9 166836                      1  
                     166837                      0  
                     166838                      1  
                     166839                      1  
                     166840                      1

**3b:** Initialize a `GradientBoostingRegressor` with the default parameters, fit it on your training set, and score it on your test set.

In [104]:
# your code here
X_train1.shape

(239673, 10)

In [105]:
y_train.shape

(239673,)

In [106]:
gbr_ord = GradientBoostingRegressor()

In [107]:
gbr_ord.fit(X_train1,y_train)

GradientBoostingRegressor()

In [108]:
gbr_ord.score(X_train1, y_train), gbr_ord.score(X_test1, y_test)

(0.169766613760161, 0.15392592049150633)

The final scores using ordinal encoding were:

* Training Data Set: 0.170
* Test Data Set: 0.154

**3c:** Repeat these same steps for the `TargetEncoder` and the `OneHotEncoder`

**Important:** The `OneHotEncoder` can take awhile to fit.  If nothing happens in around 4 minutes, just cancel the process and try it again later on when you have more time.

### Target Encoding:

In [ ]:
# your code here

In [129]:
targenc = ce.TargetEncoder(verbose=1)
targenc.fit(X_train,y_train)
X_train_t = targenc.transform(X_train)
# note - can train and fit in one step with targenc.fit_transform(X, y)
X_test_t = targenc.transform(X_test)

/usr/local/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [121]:
X_train_t.head()

id  calendar_date  day_of_week  holiday  \
id                                                                            
air_00a91d42b08b08d9 166836  25.889401      24.552180    23.021194        0   
                     166837  25.889401      26.474898    26.250407        0   
                     166838  25.889401      16.762440    17.240350        0   
                     166839  25.889401      17.473684    17.720715        0   
                     166840  25.889401      19.309524    19.231336        0   

                                 genre       area   latitude   longitude  \
id                                                                         
air_00a91d42b08b08d9 166836  22.553878  26.268049  35.694003  139.753595   
                     166837  22.553878  26.268049  35.694003  139.753595   
                     166838  22.553878  26.268049  35.694003  139.753595   
                     166839  22.553878  26.268049  35.694003  139.753595   
                     166840  22.553878  26.268049  35.694003  139.753595   

                             reserve_visitors  reserve_visitors_null  
id                                                                    
air_00a91d42b08b08d9 166836               0.0                      1  
                     166837               4.0                      0  
                     166838               0.0                      1  
                     166839               0.0                      1  
                     166840               0.0                      1

In [122]:
X_test_t.head()

id  calendar_date  day_of_week  holiday  \
id                                                                            
air_00a91d42b08b08d9 166836  25.889401      24.552180    23.021194        0   
                     166837  25.889401      26.474898    26.250407        0   
                     166838  25.889401      16.762440    17.240350        0   
                     166839  25.889401      17.473684    17.720715        0   
                     166840  25.889401      19.309524    19.231336        0   

                                 genre       area   latitude   longitude  \
id                                                                         
air_00a91d42b08b08d9 166836  22.553878  26.268049  35.694003  139.753595   
                     166837  22.553878  26.268049  35.694003  139.753595   
                     166838  22.553878  26.268049  35.694003  139.753595   
                     166839  22.553878  26.268049  35.694003  139.753595   
                     166840  22.553878  26.268049  35.694003  139.753595   

                             reserve_visitors  reserve_visitors_null  
id                                                                    
air_00a91d42b08b08d9 166836               0.0                      1  
                     166837               4.0                      0  
                     166838               0.0                      1  
                     166839               0.0                      1  
                     166840               0.0                      1

In [ ]:
# Initialise GradientBoosting to fit using target encoding.

In [123]:
gbr_targ = GradientBoostingRegressor()

In [124]:
gbr_targ.fit(X_train_t,y_train)

GradientBoostingRegressor()

In [131]:
gbr_targ.score(X_train_t, y_train), gbr_targ.score(X_test_t, y_test)

(0.48825479062698207, 0.4130945639735639)

Outputs for target encoding:
* On training data: 0.488
* On testing data: 0.413

### OneHot Encoding

In [ ]:
# Set up one hot encoder

In [132]:
onehotenc = ce.OneHotEncoder(verbose=1,use_cat_names=True)

In [133]:
X_train_onehot = onehotenc.fit_transform(X_train)

/usr/local/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [134]:
X_train_onehot.head()

id_air_00a91d42b08b08d9  id_air_0164b9927d20bcc3  \
id                                                                              
air_00a91d42b08b08d9 166836                        1                        0   
                     166837                        1                        0   
                     166838                        1                        0   
                     166839                        1                        0   
                     166840                        1                        0   

                             id_air_0241aa3964b7f861  id_air_0328696196e46f18  \
id                                                                              
air_00a91d42b08b08d9 166836                        0                        0   
                     166837                        0                        0   
                     166838                        0                        0   
                     166839                        0                        0   
                     166840                        0                        0   

                             id_air_034a3d5b40d5b1b1  id_air_036d4f1ee7285390  \
id                                                                              
air_00a91d42b08b08d9 166836                        0                        0   
                     166837                        0                        0   
                     166838                        0                        0   
                     166839                        0                        0   
                     166840                        0                        0   

                             id_air_0382c794b73b51ad  id_air_03963426c9312048  \
id                                                                              
air_00a91d42b08b08d9 166836                        0                        0   
                     166837                        0                        0   
                     166838                        0                        0   
                     166839                        0                        0   
                     166840                        0                        0   

                             id_air_04341b588bde96cd  id_air_049f6d5b402a31b2  \
id                                                                              
air_00a91d42b08b08d9 166836                        0                        0   
                     166837                        0                        0   
                     166838                        0                        0   
                     166839                        0                        0   
                     166840                        0                        0   

                             ...  area_Tōkyō-to Taitō-ku Kuramae  \
id                           ...                                   
air_00a91d42b08b08d9 166836  ...                               0   
                     166837  ...                               0   
                     166838  ...                               0   
                     166839  ...                               0   
                     166840  ...                               0   

                             area_Shizuoka-ken Numazu-shi Miyukichō  \
id                                                                    
air_00a91d42b08b08d9 166836                                       0   
                     166837                                       0   
                     166838                                       0   
                     166839                                       0   
                     166840                                       0   

                             area_Niigata-ken Nagaoka-shi Ōtedōri  \
id                                                                  
air_00a91d42b08b08d9 166836                                     0   
       

In [137]:
X_test_onehot = onehotenc.transform(X_test)

In [ ]:
# Set up gradient boosting for one hot encoding model fit

In [138]:
gbr_onehot = GradientBoostingRegressor()

In [139]:
gbr_onehot.fit(X_train_onehot, y_train)

GradientBoostingRegressor()

In [140]:
gbr_onehot.score(X_train_onehot,y_train), gbr_onehot.score(X_test_onehot,y_test)

(0.1686515772414905, 0.1716728739741108)

Output test scores using One Hot Encoding:
* Training data set: 0.167
* Test data set: 0.171

### Overall comparison of encoding methods

It appears that **target encoding** had the best impact providing a score of **0.41** for the test data, compared with scorse of 0.15 for ordinal encoding, and 0.17 for one hot encoding.
We will use the traget encoded model from here.





**Step 4:** Look at your most important features

Similar to the previous lab, take your model's most important features and load them into a dataframe to see what's driving your results.

In [141]:
# your code here

targenc = ce.TargetEncoder(verbose=1)
targenc.fit(X_train,y_train)
X_train_t = targenc.transform(X_train)
# note - can train and fit in one step with targenc.fit_transform(X, y)
X_test_t = targenc.transform(X_test)


/usr/local/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [143]:
gbr_targ = GradientBoostingRegressor()
gbr_targ.fit(X_train_t,y_train)

GradientBoostingRegressor()

In [145]:
gbr_targ.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [158]:
X_train_t.columns.to_list()


['id',
 'calendar_date',
 'day_of_week',
 'holiday',
 'genre',
 'area',
 'latitude',
 'longitude',
 'reserve_visitors',
 'reserve_visitors_null']

In [153]:
gbr_targ.feature_importances_

array([8.45829430e-01, 1.36463880e-01, 3.29724103e-03, 1.12918861e-04,
       3.10131160e-04, 1.49975942e-03, 3.83958701e-03, 8.60298212e-03,
       4.40701253e-05, 0.00000000e+00])

In [165]:
feature_df = pd.DataFrame()
feature_df['features'] = X_train_t.columns.to_list()
feature_df['importance'] = gbr_targ.feature_importances_
feature_df.sort_values(by=['importance'],ascending=False,inplace=True)

In [166]:
feature_df

,features,importance
0,id,0.845829
1,calendar_date,0.136464
7,longitude,0.008603
6,latitude,0.003840
2,day_of_week,0.003297
5,area,0.001500
4,genre,0.000310
3,holiday,0.000113
8,reserve_visitors,0.000044
9,reserve_visitors_null,0.000000


### Major drivers are:
* ID - so which restaurant it is, and if it has similar behaviour to similarly sized restaurants in terms of frequency.

* Calendar Date - This is just an incremental time series item? Perhaps it should be removed

* Everything else seems to have only a tiny impact. Will run again excluding date, and see what it looks like.

In [167]:
X_train2 = training.drop(['visitors','visit_date','calendar_date'],axis=1).copy()
y_train = training['visitors'].copy()

X_test2 = testing.drop(['visitors','visit_date','calendar_date'],axis=1).copy()
y_test = testing['visitors'].copy() 

In [171]:

targenc2 = ce.TargetEncoder(verbose=1)
X_train_t2 = targenc2.fit_transform(X_train2, y_train)
# note - can train and fit in one step with targenc.fit_transform(X, y)
X_test_t2 = targenc2.transform(X_test2)


In [172]:
gbr_targ2 = GradientBoostingRegressor()
gbr_targ2.fit(X_train_t2,y_train)

GradientBoostingRegressor()

In [173]:
gbr_targ2.score(X_train_t2, y_train), gbr_targ2.score(X_test_t2, y_test)
# Excluding calendar date improves the scores for the test data.

(0.470152660219043, 0.46643508620732876)

In [174]:
feature_df_t2 = pd.DataFrame()
feature_df_t2['features'] = X_train_t2.columns.to_list()
feature_df_t2['importance'] = gbr_targ2.feature_importances_
feature_df_t2.sort_values(by=['importance'],ascending=False,inplace=True)

In [175]:
feature_df_t2
# The breakdown of feature importance is now consistent with the results manual.

,features,importance
0,id,0.876656
1,day_of_week,0.104548
2,holiday,0.007040
7,reserve_visitors,0.004385
6,longitude,0.004308
5,latitude,0.002044
4,area,0.000664
3,genre,0.000355
8,reserve_visitors_null,0.000000


**Step 5:** Can model parameters improve your score?  

Take the **best** version of your encoding method and try changing some parameters with your model to see if it improves your score.  

You won't have a ton of time to do this, but try some of the following:

 - Try increasing the number of trees your model uses -- 250, 500, or perhaps more trees if time permits
 - Try experimenting with differing values for tree depth -- the default is 3, but perhaps 4, 5 or 6 works better
 - Try improving fitting time by introducing some **randomness** into your data with the following two model parameters:
   - `subsample`: this dictates what proportion of your data will be used for each tree.  A value of `0.7` means 70% of your data will be used for a particular tree, chosen at random
   - `max_features`: this is the portion of columns that are used at each individual split.  If you enter an integer the model will randomly select that number of columns, if you enter a decimal it will randomly select that portion of columns.
   - It can be very useful to find the most sparse model that will still give you comparable results.  Ie, if you find a gbm with 500 trees and a max_depth of 4 gives you the best results, it can be very beneficial if you can get those same results with a `subsample` value of 0.6 and a `max_features` score of 0.7, because your model will fit ~50% faster.
   
This step is open ended, so we will likely have to end class in the middle of it.

In [ ]:
# your code here

In [177]:
# Start off by looking at existing set of parameters
gbr_targ2.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'deprecated',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [178]:
gbm = GradientBoostingRegressor()

In [185]:
# we'll try doing a parameter sweep (code from Class 11, expanded slightly)
# actually makes 3*2*4*3*3 fits, so estimates 216 models!
n_estimators  = [100, 250, 500]
learning_rate = [.05, .1]
tree_depth    = [3, 4, 5, 6]
subsample = [0.5,0.7,1]
max_features = [3,6,9]
cv_scores     = []
counter = 0

for estimators in n_estimators:
    for rate in learning_rate:
        for depth in tree_depth:
            for ss in subsample:
                for mf in max_features:
                    # print(f"Fitting model for: {estimators, rate, depth, ss, mf}:")
                    gbm.set_params(n_estimators=estimators, learning_rate=rate, max_depth=depth,subsample=ss,max_features=mf)
                    gbm.fit(X_train_t2, y_train)
                    cv_scores.append((gbm.score(X_test_t2, y_test), estimators, rate, depth, ss, mf))
                    counter += 1
                    print(f"{counter}: {cv_scores[-1]}")

1: (0.4455736059375326, 100, 0.05, 3, 0.5, 3)
2: (0.4617009413143647, 100, 0.05, 3, 0.5, 6)
3: (0.461836341815839, 100, 0.05, 3, 0.5, 9)
4: (0.44384595032118923, 100, 0.05, 3, 0.7, 3)
5: (0.46227726911883826, 100, 0.05, 3, 0.7, 6)
6: (0.4609191192463431, 100, 0.05, 3, 0.7, 9)
7: (0.43922475429217445, 100, 0.05, 3, 1, 3)
8: (0.46085748471285937, 100, 0.05, 3, 1, 6)
9: (0.46097094982822806, 100, 0.05, 3, 1, 9)
10: (0.46492279351858223, 100, 0.05, 4, 0.5, 3)
11: (0.47422799335577603, 100, 0.05, 4, 0.5, 6)
12: (0.47149228474957583, 100, 0.05, 4, 0.5, 9)
13: (0.4601532236896849, 100, 0.05, 4, 0.7, 3)
14: (0.4711099658747969, 100, 0.05, 4, 0.7, 6)
15: (0.4707033819431491, 100, 0.05, 4, 0.7, 9)
16: (0.4627428979931284, 100, 0.05, 4, 1, 3)
17: (0.4694705069761348, 100, 0.05, 4, 1, 6)
18: (0.469699611461029, 100, 0.05, 4, 1, 9)
19: (0.48064169551520275, 100, 0.05, 5, 0.5, 3)
20: (0.4816095426641702, 100, 0.05, 5, 0.5, 6)
21: (0.48423605025578986, 100, 0.05, 5, 0.5, 9)
22: (0.47567803398120545, 

177: (0.5267415040066816, 500, 0.05, 6, 0.7, 9)
178: (0.520525295136127, 500, 0.05, 6, 1, 3)
179: (0.5254496485554085, 500, 0.05, 6, 1, 6)
180: (0.5250721326849422, 500, 0.05, 6, 1, 9)
181: (0.485701348876988, 500, 0.1, 3, 0.5, 3)
182: (0.4911559281715716, 500, 0.1, 3, 0.5, 6)
183: (0.49054666400307945, 500, 0.1, 3, 0.5, 9)
184: (0.48376072248045343, 500, 0.1, 3, 0.7, 3)
185: (0.49054132283349916, 500, 0.1, 3, 0.7, 6)
186: (0.4907448765146507, 500, 0.1, 3, 0.7, 9)
187: (0.48321187708430535, 500, 0.1, 3, 1, 3)
188: (0.4910858332767616, 500, 0.1, 3, 1, 6)
189: (0.4924223851575854, 500, 0.1, 3, 1, 9)
190: (0.5001959405487633, 500, 0.1, 4, 0.5, 3)
191: (0.5057487535337934, 500, 0.1, 4, 0.5, 6)
192: (0.506172805813887, 500, 0.1, 4, 0.5, 9)
193: (0.5007684968973143, 500, 0.1, 4, 0.7, 3)
194: (0.5070389169011316, 500, 0.1, 4, 0.7, 6)
195: (0.5100551450144424, 500, 0.1, 4, 0.7, 9)
196: (0.5031288948686907, 500, 0.1, 4, 1, 3)
197: (0.5128325506461242, 500, 0.1, 4, 1, 6)
198: (0.5141283163994661

In [ ]:
1: (0.4455736059375326, 100, 0.05, 3, 0.5, 3)
2: (0.4617009413143647, 100, 0.05, 3, 0.5, 6)
3: (0.461836341815839, 100, 0.05, 3, 0.5, 9)
4: (0.44384595032118923, 100, 0.05, 3, 0.7, 3)
5: (0.46227726911883826, 100, 0.05, 3, 0.7, 6)
6: (0.4609191192463431, 100, 0.05, 3, 0.7, 9)
7: (0.43922475429217445, 100, 0.05, 3, 1, 3)
8: (0.46085748471285937, 100, 0.05, 3, 1, 6)
9: (0.46097094982822806, 100, 0.05, 3, 1, 9)
10: (0.46492279351858223, 100, 0.05, 4, 0.5, 3)
11: (0.47422799335577603, 100, 0.05, 4, 0.5, 6)
12: (0.47149228474957583, 100, 0.05, 4, 0.5, 9)
13: (0.4601532236896849, 100, 0.05, 4, 0.7, 3)
14: (0.4711099658747969, 100, 0.05, 4, 0.7, 6)
15: (0.4707033819431491, 100, 0.05, 4, 0.7, 9)
16: (0.4627428979931284, 100, 0.05, 4, 1, 3)
17: (0.4694705069761348, 100, 0.05, 4, 1, 6)
18: (0.469699611461029, 100, 0.05, 4, 1, 9)
19: (0.48064169551520275, 100, 0.05, 5, 0.5, 3)
20: (0.4816095426641702, 100, 0.05, 5, 0.5, 6)
21: (0.48423605025578986, 100, 0.05, 5, 0.5, 9)
22: (0.47567803398120545, 100, 0.05, 5, 0.7, 3)
23: (0.48347480231238515, 100, 0.05, 5, 0.7, 6)
24: (0.483322565331466, 100, 0.05, 5, 0.7, 9)
25: (0.47576269891142786, 100, 0.05, 5, 1, 3)
26: (0.48152055780095604, 100, 0.05, 5, 1, 6)
27: (0.48010545423187256, 100, 0.05, 5, 1, 9)
28: (0.48711084037660846, 100, 0.05, 6, 0.5, 3)
29: (0.4906789326126998, 100, 0.05, 6, 0.5, 6)
30: (0.4933188661783352, 100, 0.05, 6, 0.5, 9)
31: (0.4873147183555254, 100, 0.05, 6, 0.7, 3)
32: (0.49463381943590135, 100, 0.05, 6, 0.7, 6)
33: (0.49321864020309736, 100, 0.05, 6, 0.7, 9)
34: (0.48813504788714523, 100, 0.05, 6, 1, 3)
35: (0.49278418211238095, 100, 0.05, 6, 1, 6)
36: (0.4908470898882775, 100, 0.05, 6, 1, 9)
37: (0.4631125985024026, 100, 0.1, 3, 0.5, 3)
38: (0.468586916707215, 100, 0.1, 3, 0.5, 6)
39: (0.4717795063246476, 100, 0.1, 3, 0.5, 9)
40: (0.4644843339942585, 100, 0.1, 3, 0.7, 3)
41: (0.46947274948439965, 100, 0.1, 3, 0.7, 6)
42: (0.4691902954372431, 100, 0.1, 3, 0.7, 9)
43: (0.46256235883652397, 100, 0.1, 3, 1, 3)
44: (0.4701263172629937, 100, 0.1, 3, 1, 6)
45: (0.4664350862073289, 100, 0.1, 3, 1, 9)
46: (0.4749433848402662, 100, 0.1, 4, 0.5, 3)
47: (0.4800256737066454, 100, 0.1, 4, 0.5, 6)
48: (0.4801441247795113, 100, 0.1, 4, 0.5, 9)
49: (0.4804292805347694, 100, 0.1, 4, 0.7, 3)
50: (0.48108756669176445, 100, 0.1, 4, 0.7, 6)
51: (0.4812020664447897, 100, 0.1, 4, 0.7, 9)
52: (0.4743217733085582, 100, 0.1, 4, 1, 3)
53: (0.48267782412775484, 100, 0.1, 4, 1, 6)
54: (0.4823515923268735, 100, 0.1, 4, 1, 9)
55: (0.48852700594255805, 100, 0.1, 5, 0.5, 3)
56: (0.49377120696911647, 100, 0.1, 5, 0.5, 6)
57: (0.4972244412537383, 100, 0.1, 5, 0.5, 9)
58: (0.49010814990878115, 100, 0.1, 5, 0.7, 3)
59: (0.49462752672776933, 100, 0.1, 5, 0.7, 6)
60: (0.49672544413346653, 100, 0.1, 5, 0.7, 9)
61: (0.4880855868256234, 100, 0.1, 5, 1, 3)
62: (0.4924677106108971, 100, 0.1, 5, 1, 6)
63: (0.4920956823510777, 100, 0.1, 5, 1, 9)
64: (0.5009380681932547, 100, 0.1, 6, 0.5, 3)
65: (0.5059074322309379, 100, 0.1, 6, 0.5, 6)
66: (0.5101843556275107, 100, 0.1, 6, 0.5, 9)
67: (0.4999031949424322, 100, 0.1, 6, 0.7, 3)
68: (0.5080295526571504, 100, 0.1, 6, 0.7, 6)
69: (0.5120148994738146, 100, 0.1, 6, 0.7, 9)
70: (0.500289149284443, 100, 0.1, 6, 1, 3)
71: (0.5037770949868767, 100, 0.1, 6, 1, 6)
72: (0.5134143267670113, 100, 0.1, 6, 1, 9)
73: (0.46793035201722033, 250, 0.05, 3, 0.5, 3)
74: (0.4730380224492532, 250, 0.05, 3, 0.5, 6)
75: (0.47306633850142854, 250, 0.05, 3, 0.5, 9)
76: (0.4650593235854288, 250, 0.05, 3, 0.7, 3)
77: (0.4707075345262901, 250, 0.05, 3, 0.7, 6)
78: (0.47221404691337354, 250, 0.05, 3, 0.7, 9)
79: (0.46706888767901367, 250, 0.05, 3, 1, 3)
80: (0.46919911840518636, 250, 0.05, 3, 1, 6)
81: (0.46832991853892647, 250, 0.05, 3, 1, 9)
82: (0.477950297112983, 250, 0.05, 4, 0.5, 3)
83: (0.4842485856480917, 250, 0.05, 4, 0.5, 6)
84: (0.4862512557279436, 250, 0.05, 4, 0.5, 9)
85: (0.4805385120657285, 250, 0.05, 4, 0.7, 3)
86: (0.4855068529510287, 250, 0.05, 4, 0.7, 6)
87: (0.4877179474810541, 250, 0.05, 4, 0.7, 9)
88: (0.4764089030877755, 250, 0.05, 4, 1, 3)
89: (0.48211024469066555, 250, 0.05, 4, 1, 6)
90: (0.48201502697188836, 250, 0.05, 4, 1, 9)
91: (0.49314221463942176, 250, 0.05, 5, 0.5, 3)
92: (0.4953179520545111, 250, 0.05, 5, 0.5, 6)
93: (0.5009374865482524, 250, 0.05, 5, 0.5, 9)
94: (0.492918255285025, 250, 0.05, 5, 0.7, 3)
95: (0.5003872285049047, 250, 0.05, 5, 0.7, 6)
96: (0.5015434944560491, 250, 0.05, 5, 0.7, 9)
97: (0.49312494007089314, 250, 0.05, 5, 1, 3)
98: (0.49436643397868596, 250, 0.05, 5, 1, 6)
99: (0.501810735611103, 250, 0.05, 5, 1, 9)
100: (0.5042595692670995, 250, 0.05, 6, 0.5, 3)
101: (0.5110230435364713, 250, 0.05, 6, 0.5, 6)
102: (0.5133264595664504, 250, 0.05, 6, 0.5, 9)
103: (0.5059980775522201, 250, 0.05, 6, 0.7, 3)
104: (0.5126824068347866, 250, 0.05, 6, 0.7, 6)
105: (0.5168493677946928, 250, 0.05, 6, 0.7, 9)
106: (0.5072183408452697, 250, 0.05, 6, 1, 3)
107: (0.5141534299440261, 250, 0.05, 6, 1, 6)
108: (0.5125115047221878, 250, 0.05, 6, 1, 9)
109: (0.4731115123563393, 250, 0.1, 3, 0.5, 3)
110: (0.4820752331650776, 250, 0.1, 3, 0.5, 6)
111: (0.4832296424363173, 250, 0.1, 3, 0.5, 9)
112: (0.47477655026496923, 250, 0.1, 3, 0.7, 3)
113: (0.4831853188176506, 250, 0.1, 3, 0.7, 6)
114: (0.4839901002256045, 250, 0.1, 3, 0.7, 9)
115: (0.4741932269226665, 250, 0.1, 3, 1, 3)
116: (0.4791627604053964, 250, 0.1, 3, 1, 6)
117: (0.48056350627485633, 250, 0.1, 3, 1, 9)
118: (0.48996510355035827, 250, 0.1, 4, 0.5, 3)
119: (0.4977670991936797, 250, 0.1, 4, 0.5, 6)
120: (0.4973095703984205, 250, 0.1, 4, 0.5, 9)
121: (0.48999307117526103, 250, 0.1, 4, 0.7, 3)
122: (0.49795644027104546, 250, 0.1, 4, 0.7, 6)
123: (0.5009725901906668, 250, 0.1, 4, 0.7, 9)
124: (0.48935104542082575, 250, 0.1, 4, 1, 3)
125: (0.4968915345055499, 250, 0.1, 4, 1, 6)
126: (0.49744460624402953, 250, 0.1, 4, 1, 9)
127: (0.5000349446734895, 250, 0.1, 5, 0.5, 3)
128: (0.508173142578243, 250, 0.1, 5, 0.5, 6)
129: (0.5103770232720634, 250, 0.1, 5, 0.5, 9)
130: (0.504304721556353, 250, 0.1, 5, 0.7, 3)
131: (0.5116385104223693, 250, 0.1, 5, 0.7, 6)
132: (0.5127127182720297, 250, 0.1, 5, 0.7, 9)
133: (0.5075389343013335, 250, 0.1, 5, 1, 3)
134: (0.5135918018647703, 250, 0.1, 5, 1, 6)
135: (0.5144605513106575, 250, 0.1, 5, 1, 9)
136: (0.5145151419487928, 250, 0.1, 6, 0.5, 3)
137: (0.5191997012336755, 250, 0.1, 6, 0.5, 6)
138: (0.5154453597886919, 250, 0.1, 6, 0.5, 9)
139: (0.5132648184332326, 250, 0.1, 6, 0.7, 3)
140: (0.5223343134683036, 250, 0.1, 6, 0.7, 6)
141: (0.5235515019115297, 250, 0.1, 6, 0.7, 9)
142: (0.5218393396105337, 250, 0.1, 6, 1, 3)
143: (0.5285178896137217, 250, 0.1, 6, 1, 6)
144: (0.5283994950563423, 250, 0.1, 6, 1, 9)
145: (0.47584446408189873, 500, 0.05, 3, 0.5, 3)
146: (0.4816454363710784, 500, 0.05, 3, 0.5, 6)
147: (0.481931267831033, 500, 0.05, 3, 0.5, 9)
148: (0.47711805063905466, 500, 0.05, 3, 0.7, 3)
149: (0.48275431860664, 500, 0.05, 3, 0.7, 6)
150: (0.48215287425520237, 500, 0.05, 3, 0.7, 9)
151: (0.47461788758906476, 500, 0.05, 3, 1, 3)
152: (0.47779353845230443, 500, 0.05, 3, 1, 6)
153: (0.4776341335549237, 500, 0.05, 3, 1, 9)
154: (0.4918057220353004, 500, 0.05, 4, 0.5, 3)
155: (0.49864428360151003, 500, 0.05, 4, 0.5, 6)
156: (0.49816453464775123, 500, 0.05, 4, 0.5, 9)
157: (0.4915434472963226, 500, 0.05, 4, 0.7, 3)
158: (0.4965216858197258, 500, 0.05, 4, 0.7, 6)
159: (0.49990304750388415, 500, 0.05, 4, 0.7, 9)
160: (0.4909861724229809, 500, 0.05, 4, 1, 3)
161: (0.49663223416275337, 500, 0.05, 4, 1, 6)
162: (0.4968909145780128, 500, 0.05, 4, 1, 9)
163: (0.5042082714016661, 500, 0.05, 5, 0.5, 3)
164: (0.5098146630275644, 500, 0.05, 5, 0.5, 6)
165: (0.5106050336896822, 500, 0.05, 5, 0.5, 9)
166: (0.5073298490728723, 500, 0.05, 5, 0.7, 3)
167: (0.5135795141432307, 500, 0.05, 5, 0.7, 6)
168: (0.5138290515203203, 500, 0.05, 5, 0.7, 9)
169: (0.5055146252857574, 500, 0.05, 5, 1, 3)
170: (0.5127174084353578, 500, 0.05, 5, 1, 6)
171: (0.5145452299165352, 500, 0.05, 5, 1, 9)
172: (0.5157906300748032, 500, 0.05, 6, 0.5, 3)
173: (0.5204395233098085, 500, 0.05, 6, 0.5, 6)
174: (0.5197243863824426, 500, 0.05, 6, 0.5, 9)
175: (0.5179947574936732, 500, 0.05, 6, 0.7, 3)
176: (0.5235895818885006, 500, 0.05, 6, 0.7, 6)
177: (0.5267415040066816, 500, 0.05, 6, 0.7, 9)
178: (0.520525295136127, 500, 0.05, 6, 1, 3)
179: (0.5254496485554085, 500, 0.05, 6, 1, 6)
180: (0.5250721326849422, 500, 0.05, 6, 1, 9)
181: (0.485701348876988, 500, 0.1, 3, 0.5, 3)
182: (0.4911559281715716, 500, 0.1, 3, 0.5, 6)
183: (0.49054666400307945, 500, 0.1, 3, 0.5, 9)
184: (0.48376072248045343, 500, 0.1, 3, 0.7, 3)
185: (0.49054132283349916, 500, 0.1, 3, 0.7, 6)
186: (0.4907448765146507, 500, 0.1, 3, 0.7, 9)
187: (0.48321187708430535, 500, 0.1, 3, 1, 3)
188: (0.4910858332767616, 500, 0.1, 3, 1, 6)
189: (0.4924223851575854, 500, 0.1, 3, 1, 9)
190: (0.5001959405487633, 500, 0.1, 4, 0.5, 3)
191: (0.5057487535337934, 500, 0.1, 4, 0.5, 6)
192: (0.506172805813887, 500, 0.1, 4, 0.5, 9)
193: (0.5007684968973143, 500, 0.1, 4, 0.7, 3)
194: (0.5070389169011316, 500, 0.1, 4, 0.7, 6)
195: (0.5100551450144424, 500, 0.1, 4, 0.7, 9)
196: (0.5031288948686907, 500, 0.1, 4, 1, 3)
197: (0.5128325506461242, 500, 0.1, 4, 1, 6)
198: (0.5141283163994661, 500, 0.1, 4, 1, 9)
199: (0.5070441688349315, 500, 0.1, 5, 0.5, 3)
200: (0.5187243099481371, 500, 0.1, 5, 0.5, 6)
201: (0.514094159031942, 500, 0.1, 5, 0.5, 9)
202: (0.5128052140003585, 500, 0.1, 5, 0.7, 3)
203: (0.5210556315361288, 500, 0.1, 5, 0.7, 6)
204: (0.5196319217836294, 500, 0.1, 5, 0.7, 9)
205: (0.5208237007890462, 500, 0.1, 5, 1, 3)
206: (0.5278452580764665, 500, 0.1, 5, 1, 6)
207: (0.5280767844094478, 500, 0.1, 5, 1, 9)
208: (0.5194528837093888, 500, 0.1, 6, 0.5, 3)
209: (0.5211767442102149, 500, 0.1, 6, 0.5, 6)
210: (0.5212837099905412, 500, 0.1, 6, 0.5, 9)
211: (0.5213901715331577, 500, 0.1, 6, 0.7, 3)
212: (0.5260135117633915, 500, 0.1, 6, 0.7, 6)
213: (0.5232979849226816, 500, 0.1, 6, 0.7, 9)
214: (0.5309642856246088, 500, 0.1, 6, 1, 3)
215: (0.5329311109783008, 500, 0.1, 6, 1, 6)
216: (0.5314454102793863, 500, 0.1, 6, 1, 9)
